<a href="https://colab.research.google.com/github/dariiimo/nlp2021/blob/main/Lab3/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/нлп/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [8]:
RL_file = open(folder+'sentences.txt')
RL_sentences = RL_file.read().split('. ')[:-1]

DARINA_file = open(folder+'100.txt', encoding='windows-1251')
DARINA_sentences = DARINA_file.read().split('. ')[:-1]

RL_sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

In [10]:
sent_df = []

for sent in RL_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in DARINA_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning,0
1,This is what Google DeepMind did with its popu...,Reinforcement Learning,0
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
4,The agent collects information about its curre...,Reinforcement Learning,0
...,...,...,...
211,This includes balancing the prospective benefi...,Neural network module for recognizing brain di...,1
212,"Users report their symptoms into the app, whic...",Neural network module for recognizing brain di...,1
213,Most common users could be people with serious...,Neural network module for recognizing brain di...,1
214,Body scans can spot cancer and vascular diseas...,Neural network module for recognizing brain di...,1


## Здійснити підготовку набору даних до моделювання (tokenization&embedding, train_test_split)


In [13]:
max_tokens = 10000

sentences = RL_sentences+DARINA_sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens

23

#### Tokenization&Embedding

In [14]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(216, 23), dtype=int64, numpy=
array([[ 108,    6,  340, ...,  120,    4, 1214],
       [  27,    8,   47, ...,    6,   76,    9],
       [   3,  152,   26, ...,  618,   22,  216],
       ...,
       [  86,  314,  342, ...,    7,   64,  126],
       [1387,  794,   20, ..., 1159,    0,    0],
       [ 244,    2,  119, ...,  446,    8,    2]])>

In [15]:
embedding = layers.Embedding(input_dim=max_tokens, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=avg_tokens) # how long is each input

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(216, 23, 128), dtype=float32, numpy=
array([[[ 1.92381404e-02, -2.40186695e-02,  4.15458791e-02, ...,
         -3.67470756e-02,  1.22135282e-02,  9.77214426e-03],
        [ 3.28471512e-03, -2.82566622e-03,  1.81098096e-02, ...,
         -2.93680076e-02, -3.60281467e-02,  1.00275129e-03],
        [-2.98740864e-02, -2.30908990e-02,  5.57623804e-04, ...,
          1.05183125e-02,  1.23597160e-02, -3.08249146e-03],
        ...,
        [ 7.85101578e-03, -2.05122828e-02, -1.97264310e-02, ...,
          2.34546550e-02, -3.78147252e-02, -3.35229039e-02],
        [-3.97031307e-02,  4.40392233e-02,  1.06388442e-02, ...,
          4.68807332e-02,  2.30207928e-02, -1.11668482e-02],
        [ 1.01379864e-02,  2.05031671e-02,  4.65751328e-02, ...,
         -2.36142166e-02, -3.19918394e-02, -2.08856594e-02]],

       [[ 2.22262181e-02,  1.42224878e-03, -3.57791409e-02, ...,
         -1.09992512e-02,  4.66940664e-02,  2.25063078e-02],
        [ 1.37925260e-02, -2.97180414e-02,  1.7

#### Train/Test Split

In [16]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.9, test_size=0.1)
train_split

,Sentence,Target,Label
135,"By sifting out healthy patients, the system al...",Neural network module for recognizing brain di...,1
88,Early research of deep reinforcement learning ...,Reinforcement Learning,0
103,"In many reinforcement learning problems, the a...",Reinforcement Learning,0
124,One valuable approach for improving medical ca...,Neural network module for recognizing brain di...,1
67,in usual circumstances we would require an aut...,Reinforcement Learning,0
...,...,...,...
152,The AI technology has been applied to various ...,Neural network module for recognizing brain di...,1
118,Phenotypes such as this are hard to define in ...,Neural network module for recognizing brain di...,1
104,"Based on the feedback it receives, the agent l...",Reinforcement Learning,0
142,"Instead, AI systems will be used to highlight ...",Neural network module for recognizing brain di...,1


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: Naive Bayes Model, Dense Neural Network

In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_0.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 90.91%


### Dense Neural Network

In [20]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model


In [22]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [24]:
# Fit the model
model_1_history = model_1.fit(train_split['Sentence'], # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_split['Label'],
                              epochs=50,
                              validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0559 - accuracy: 1.0000 - val_loss: 0.2501 - val_accuracy: 0.9545
Epoch 2/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0536 - accuracy: 1.0000 - val_loss: 0.2469 - val_accuracy: 0.9545
Epoch 3/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0515 - accuracy: 1.0000 - val_loss: 0.2439 - val_accuracy: 0.9545
Epoch 4/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0495 - accuracy: 1.0000 - val_loss: 0.2419 - val_accuracy: 0.9545
Epoch 5/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0475 - accuracy: 1.0000 - val_loss: 0.2435 - val_accuracy: 0.9545
Epoch 6/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0459 - accuracy: 1.0000 - val_loss: 0.2438 - val_accuracy: 0.9545
Epoch 7/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0440 - accuracy: 1.0000 - val_loss: 0.2413 - val_accuracy: 0.9545
Epoch 8/50
7/7 [===========

## Здійснити порівняння ефективності моделей

In [25]:
calculate_results(test_split['Label'], model_0.predict(test_split['Sentence']))

{'accuracy': 90.9090909090909,
 'f1': 0.9066558441558442,
 'precision': 0.9212121212121211,
 'recall': 0.9090909090909091}

In [26]:
calculate_results(test_split['Label'], tf.squeeze(tf.round(model_1.predict(test_split['Sentence']))))


{'accuracy': 95.45454545454545,
 'f1': 0.9540503069914834,
 'precision': 0.9577922077922078,
 'recall': 0.9545454545454546}